<a href="https://www.kaggle.com/alejandroerestrepo/categorical-variables-intermediate-ml-session-3?scriptVersionId=89853048" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### In this short notebook I use the Knowledge we have learnt until the session "Categorical Variables" of the course: Intermediate Machine Learning. 
#### I use the concepts of Imputer (Dealing with missing values in numerical columns) and also I use One-Hot Encoding technique for dealing with Categorical Variables. 
#### Note that when I perform One-Hot Encoding, I idenfity the columns with missing values in both datasets X_test and X (Which is the trainig dataset), so I remove all columns with missing values to reduce future problems. 

Final note: I'm a beginer with ML Projects, I'm sorry if I made a mistake. I hope you like and can well use this framework to better understand our data and our firsts techniques. 

Greetings from Colombia ! 

### Getting the data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

### Split data into Categorical & Numerical columns. 

In [2]:
#Save numerical only columns from X & X_test
num_X_test = X_test.select_dtypes(exclude=['object'])
num_X = X.select_dtypes(exclude=['object'])

In [3]:
#Generate 2 lists with the names of categorical columns 
cat_cols_X_test = [col for col in X_test.columns if X_test[col].dtype == "object"]
cat_cols_X = [col for col in X_test.columns if X_test[col].dtype == "object"]

#Save Categorical columns from X & X_test
cat_X_test = X_test[cat_cols_X_test]
cat_X = X[cat_cols_X]

### Impute, with the mean, the numerical only datasets (X & X_test)

In [4]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer()
imputed_num_X_test = pd.DataFrame(imputer.fit_transform(num_X_test))
imputed_num_X = pd.DataFrame(imputer.transform(num_X))

#Imputation removed column names; put them back
imputed_num_X_test.columns = num_X_test.columns
imputed_num_X.columns = num_X.columns

### Delete from the categorical Data Sets the columns with missing values in one or another dataset (cat_X_test & cat_X)

In [5]:
# bring columns names with missing values in any dataset 
cols_with_missing = [col for col in cat_X.columns if ((cat_X[col].isnull().any()) | (cat_X_test[col].isnull().any()))] 

# Drop the missing columns from the categorical DataSets
drop_cat_X_test = cat_X_test.drop(cols_with_missing, axis=1)
drop_cat_X = cat_X.drop(cols_with_missing, axis=1)

### Let's One-Hot Encode the Categorical DataSets (drop_cat_X_test & drop_cat_X)

In [6]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
OH_X = pd.DataFrame(OH_encoder.fit_transform(drop_cat_X)) # Your code here
OH_X_test = pd.DataFrame(OH_encoder.transform(drop_cat_X_test)) # Your code here

#One Hot enconder removed index, let's put it again
OH_X_test.index = imputed_num_X_test.index 
OH_X.index = imputed_num_X.index

#Add one-hot encoded columns to numerical features
OH_X_test = pd.concat([imputed_num_X_test, OH_X_test], axis=1)
OH_X = pd.concat([imputed_num_X, OH_X], axis=1)

### Fit & Predict using a Random Forest Regressor

In [7]:
#Fit the random forest model 
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(OH_X, y)
preds_test = model.predict(OH_X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


### Generate Predictions in 'submission.csv'

In [8]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)